In [11]:
import numpy as np
import tensorflow as tf
import os
from scipy.special import softmax

In [12]:
relu = lambda x: x * (x > 0)

In [13]:
def block_matvec_mul(mat_A, vec_B, BLOCK_WIDTH, BLOCK_HEIGHT, error_profile):
    gridydim = int(np.ceil(mat_A.shape[0]/BLOCK_HEIGHT))
    gridxdim = int(np.ceil(mat_A.shape[1]/BLOCK_WIDTH))
    print("GRID: (",gridydim,",",gridxdim,")")

    block_result = np.zeros((gridydim, gridxdim,BLOCK_HEIGHT ))
    error_profile = error_profile.reshape(gridydim, gridxdim,BLOCK_HEIGHT)
    assert block_result.shape == error_profile.shape, "Gridblock dimensions and error profile dimensions do not match."
    for x in range(gridxdim):
        for y in range(gridydim):
            block_result[y][x] = np.matmul(mat_A[y*BLOCK_HEIGHT:(y+1)*BLOCK_HEIGHT,
                                                 x*BLOCK_WIDTH:(x+1)*BLOCK_WIDTH], vec_B[x*BLOCK_WIDTH:(x+1)*BLOCK_WIDTH]).squeeze()
            block_result[y][x] += error_profile[y][x]

    # sum columnwise i.e., sum elements in each row i.e., axis = 1
    block_sum = block_result.sum(axis=1)

    #flatten to single vector
    product = block_sum.reshape(-1,1)

    return product

In [14]:
# Combine test and train images together into one dataset

(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
train_images = train_images.astype(np.float32) / 255.0
test_images = test_images.astype(np.float32) / 255.0  

all_images =np.concatenate([train_images, test_images], axis=0)
all_labels =np.concatenate([train_labels, test_labels], axis=0)

In [15]:
# Arguments

layer_width = 512
model_seed = 52233264

In [16]:
# load weights
tag = "mnist_dense" + '-w' + str(layer_width) + 'x' + str(layer_width) + '-' + str(model_seed)
cur_folder = os.getcwd()
weights_folder = os.path.join(cur_folder,"weights",tag)
weight_filename_tag = os.path.join(weights_folder, tag + "_")

# Layer_0
dense_weights_file = weight_filename_tag + "dense_weights.npy"
dense_biases_file = weight_filename_tag + "dense_biases.npy"
dense_weights = np.load(dense_weights_file)
dense_biases = np.load(dense_biases_file).reshape(-1,1)

# Layer_1
dense_1_weights_file = weight_filename_tag + "dense_1_weights.npy"
dense_1_biases_file = weight_filename_tag + "dense_1_biases.npy"
dense_1_weights = np.load(dense_1_weights_file)
dense_1_biases = np.load(dense_1_biases_file).reshape(-1,1)

#Layer_2
dense_2_weights_file = weight_filename_tag + "dense_2_weights.npy"
dense_2_biases_file = weight_filename_tag + "dense_2_biases.npy"
dense_2_weights = np.load(dense_2_weights_file)
dense_2_biases = np.load(dense_2_biases_file).reshape(-1,1)

In [17]:
# load input
image_no = 69999;
input_image = all_images[image_no]
input_image_label = all_labels[image_no]
print("IMAGE_NUMBER: ", image_no)
print("TRUTH_LABEL: ", input_image_label)

IMAGE_NUMBER:  69999
TRUTH_LABEL:  6


In [18]:
error_profile = np.load("./data/uniform_20210907.npy")

In [19]:
# flatten input

flattened_input = input_image.reshape(-1,1)

In [20]:
# send through Layer_0

# matvec multiplication
# l0 = np.matmul(dense_weights.transpose(), 
#                flattened_input)
l0 = block_matvec_mul(dense_weights.transpose(),
                      flattened_input,
                      BLOCK_WIDTH=98,
                      BLOCK_HEIGHT=64,
                      error_profile=error_profile)

# add bias
b0 = l0 + dense_biases

# ReLU
b0_relu = relu(b0)

GRID: ( 8 , 8 )


In [21]:
# send through Layer_1

# matvec multiplication
# l1 = np.matmul(dense_1_weights.transpose(), 
#                b0_relu)
l1 = block_matvec_mul(dense_1_weights.transpose(), 
                         b0_relu,
                         BLOCK_WIDTH=64,
                         BLOCK_HEIGHT=64,
                     error_profile=error_profile)

# add bias
b1 = l1 + dense_1_biases

# ReLU
b1_relu = relu(b1)

GRID: ( 8 , 8 )


In [24]:
# send through Layer_2

# matvec multiplication
# l2 = np.matmul(dense_2_weights.transpose(), 
#                b1_relu)
l2 = block_matvec_mul(dense_2_weights.transpose(), 
                     b1_relu,
                     BLOCK_WIDTH=64,
                     BLOCK_HEIGHT=10,
                     error_profile=error_profile[0:10,:])

# add bias
b2 = l2 + dense_2_biases

# softmax
b2_softmax = softmax(b2)

GRID: ( 1 , 8 )


ValueError: cannot reshape array of size 640 into shape (1,8,10)

In [ ]:
# Output
prediction = np.argmax(b2_softmax)
print("PREDICTION: ", prediction)